In [2]:
import pandas as pd
from pandas import read_excel

file_name = 'target_data.xlsx' 
target_data = read_excel(file_name)
target_data = target_data.fillna(0)

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
feature_cols = list(target_data.columns[1:-6])

X_all = target_data[feature_cols]

In [4]:
from sklearn.ensemble import RandomForestClassifier
import pickle

ccmodel = pickle.load(open('ccmodel.pkl', 'rb'))

prediction = ccmodel.predict(X_all)
sales_cols = list(target_data.columns[:-6])
result = target_data[sales_cols]
result["cc_sales_prediction"] = prediction
result = result[result.cc_sales_prediction == 1]
print(result.shape[0])
print(result[:1000])
result.to_excel("cc_sales_prediction.xlsx", index=False)

285
      Client  Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  \
18        19  1.0   46      94         1       1.0       0.0        0.0   
19        20  0.0   27     181         1       1.0       0.0        0.0   
20        21  1.0   24      60         1       1.0       0.0        0.0   
21        22  0.0   54       0         1       0.0       0.0        0.0   
28        29  1.0   60     150         1       0.0       0.0        1.0   
...      ...  ...  ...     ...       ...       ...       ...        ...   
1567    1568  1.0   40      17         1       0.0       0.0        0.0   
1570    1571  0.0   39     211         1       0.0       0.0        0.0   
1572    1573  1.0   69     181         1       1.0       0.0        1.0   
1587    1588  0.0   61      37         1       0.0       0.0        1.0   
1599    1600  1.0   44     200         1       1.0       0.0        1.0   

      Count_CC  Count_CL  ...  VolumeDeb_CA  VolumeDebCash_Card  \
18         0.0       0.0  ..

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [5]:
result

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,cc_sales_prediction
18,19,1.0,46,94,1,1.0,0.0,0.0,0.0,0.0,...,148.446786,0.000000,0.000000,144.125357,5.0,5.0,0.0,0.0,4.0,1
19,20,0.0,27,181,1,1.0,0.0,0.0,0.0,0.0,...,221.428571,107.142857,10.714286,103.571429,7.0,7.0,3.0,1.0,3.0,1
20,21,1.0,24,60,1,1.0,0.0,0.0,0.0,0.0,...,505.653571,178.571429,259.189286,64.285714,22.0,22.0,4.0,14.0,3.0,1
21,22,0.0,54,0,1,0.0,0.0,0.0,0.0,0.0,...,3587.078214,1392.682143,466.840000,1273.107143,29.0,29.0,4.0,12.0,9.0,1
28,29,1.0,60,150,1,0.0,0.0,1.0,0.0,0.0,...,192.621786,92.857143,0.000000,89.285714,8.0,4.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,1568,1.0,40,17,1,0.0,0.0,0.0,0.0,0.0,...,175.439286,28.571429,13.867857,125.857143,9.0,9.0,2.0,4.0,2.0,1
1570,1571,0.0,39,211,1,0.0,0.0,0.0,0.0,0.0,...,2.678571,0.000000,0.000000,0.000000,2.0,2.0,0.0,0.0,0.0,1
1572,1573,1.0,69,181,1,1.0,0.0,1.0,1.0,0.0,...,1227.413929,125.000000,256.838929,806.035714,46.0,19.0,6.0,18.0,8.0,1
1587,1588,0.0,61,37,1,0.0,0.0,1.0,0.0,0.0,...,488.821429,357.142857,67.464286,60.571429,7.0,7.0,1.0,1.0,3.0,1
